# ***I am runing the model in Google Colab***

# **Step 1: Install Required Libraries**

In [2]:
# Run this in your terminal or Colab notebook
!pip install scikit-learn sentence-transformers pandas


# **Upload files**

In [3]:
from google.colab import files

# Upload the file
uploaded = files.upload()

# Confirm the uploaded file
print(uploaded)

Saving products2.csv to products2.csv
{'products2.csv': b'product_id,name,description,material,certification\r\n1,Organic Cotton T-shirt,T-shirt made from 100% organic cotton with eco-friendly dyes.,Organic Cotton,Fair Trade\r\n2,Bamboo Cotton T-shirt,T-shirt made from a blend of bamboo and cotton fibers.,Bamboo Cotton Blend,Eco-Certified\r\n3,Hemp Fabric T-shirt,Lightweight T-shirt made from sustainable hemp fabric.,Hemp Fabric,Global Organic Textile Standard\r\n4,Recycled Cotton T-shirt,Soft T-shirt made from recycled cotton materials.,Recycled Cotton,Global Recycle Standard\r\n5,Eco-friendly Wool Sweater,Cozy wool sweater made with eco-friendly processes.,Eco-friendly Wool,Responsible Wool Standard\r\n6,Organic Wool Sweater,Warm sweater made from organic wool certified fair trade.,Organic Wool,Fair Trade Certified\r\n7,Recycled Polyester Jacket,Lightweight jacket made with recycled polyester materials.,Recycled Polyester,Global Recycle Standard\r\n8,Sustainable Denim Jacket,Denim ja

# **Step 2: Import Libraries and Load Dataset**

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

# Load the dataset
file_path = "products2.csv"
products = pd.read_csv(file_path)

# Display the first few rows
print(products.head())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


   product_id                       name  \
0           1     Organic Cotton T-shirt   
1           2      Bamboo Cotton T-shirt   
2           3        Hemp Fabric T-shirt   
3           4    Recycled Cotton T-shirt   
4           5  Eco-friendly Wool Sweater   

                                         description             material  \
0  T-shirt made from 100% organic cotton with eco...       Organic Cotton   
1  T-shirt made from a blend of bamboo and cotton...  Bamboo Cotton Blend   
2  Lightweight T-shirt made from sustainable hemp...          Hemp Fabric   
3  Soft T-shirt made from recycled cotton materials.      Recycled Cotton   
4  Cozy wool sweater made with eco-friendly proce...    Eco-friendly Wool   

                     certification  
0                       Fair Trade  
1                    Eco-Certified  
2  Global Organic Textile Standard  
3          Global Recycle Standard  
4        Responsible Wool Standard  


# **Step 3: Create Features for Embeddings**

In [5]:
# Combine columns into a features column
products['features'] = (
    products['description'] + " " +
    products['material'] + " " +
    products['certification']
)

# Display a sample
print(products[['name', 'features']].head())


                        name  \
0     Organic Cotton T-shirt   
1      Bamboo Cotton T-shirt   
2        Hemp Fabric T-shirt   
3    Recycled Cotton T-shirt   
4  Eco-friendly Wool Sweater   

                                            features  
0  T-shirt made from 100% organic cotton with eco...  
1  T-shirt made from a blend of bamboo and cotton...  
2  Lightweight T-shirt made from sustainable hemp...  
3  Soft T-shirt made from recycled cotton materia...  
4  Cozy wool sweater made with eco-friendly proce...  


# **Step 4: Generate Sentence Embeddings**

In [6]:
# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the features
embeddings = model.encode(products['features'].tolist(), show_progress_bar=True)

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(embeddings)

# Display a snippet of the similarity matrix
print("Cosine Similarity Matrix (snippet):")
print(cosine_sim[:5, :5])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine Similarity Matrix (snippet):
[[1.0000005  0.77554    0.6927787  0.7049973  0.51547676]
 [0.77554    1.0000001  0.65610087 0.71961415 0.49818957]
 [0.6927787  0.65610087 1.0000001  0.66226315 0.48709124]
 [0.7049973  0.71961415 0.66226315 1.0000001  0.43535143]
 [0.51547676 0.49818957 0.48709124 0.43535143 1.0000002 ]]


# **Step 5: Apply Clustering for Diversity**

In [7]:
# Perform KMeans clustering on the embeddings
num_clusters = 5  # Define the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
products['cluster'] = kmeans.fit_predict(embeddings)

# Display cluster assignments
print(products[['name', 'cluster']].sort_values(by='cluster'))


                            name  cluster
9         Recycled Plastic Jeans        0
4      Eco-friendly Wool Sweater        0
6      Recycled Polyester Jacket        0
7       Sustainable Denim Jacket        0
16    Compostable Dinnerware Set        1
15     Biodegradable Cutlery Set        1
12        Compostable Travel Mug        1
8           Organic Cotton Jeans        2
0         Organic Cotton T-shirt        2
3        Recycled Cotton T-shirt        2
2            Hemp Fabric T-shirt        2
1          Bamboo Cotton T-shirt        2
5           Organic Wool Sweater        2
11          Recycled Plastic Mug        3
13  Stainless Steel Water Bottle        3
14   Reusable Glass Water Bottle        3
18   Reclaimed Wood Coffee Table        4
10                    Bamboo Mug        4
17   Reclaimed Wood Dining Table        4
19             Eco-friendly Sofa        4


# **Step 6: Recommendation Functions**

# Content-Based Recommendations

In [8]:
def get_recommendations(product_name, cosine_sim, products, top_n=5):
    # Find the index of the product
    idx = products.index[products['name'] == product_name].tolist()[0]

    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort by similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top_n recommendations (excluding the product itself)
    sim_scores = sim_scores[1:top_n+1]

    # Fetch product names
    recommended_products = [products.iloc[i[0]]['name'] for i in sim_scores]
    return recommended_products

# Test the function
recommendations = get_recommendations("Organic Cotton T-shirt", cosine_sim, products)
print("Recommendations for 'Organic Cotton T-shirt':", recommendations)


Recommendations for 'Organic Cotton T-shirt': ['Bamboo Cotton T-shirt', 'Recycled Cotton T-shirt', 'Hemp Fabric T-shirt', 'Organic Wool Sweater', 'Organic Cotton Jeans']


# Testing of Content-Based Recommendations System

In [10]:
recommendations = get_recommendations("Biodegradable Cutlery Set", cosine_sim, products)
print("Recommendations for 'Biodegradable Cutlery Set':", recommendations)


Recommendations for 'Biodegradable Cutlery Set': ['Compostable Dinnerware Set', 'Compostable Travel Mug', 'Recycled Plastic Mug', 'Reclaimed Wood Dining Table', 'Reclaimed Wood Coffee Table']


# Clustered Recommendations

In [11]:
def get_clustered_recommendations(product_name, cosine_sim, products, top_n=5):
    idx = products.index[products['name'] == product_name].tolist()[0]
    target_cluster = products.iloc[idx]['cluster']

    # Filter products in the same cluster
    same_cluster = products[products['cluster'] == target_cluster]

    # Get similarity scores within the cluster
    sim_scores = [(i, cosine_sim[idx][i]) for i in same_cluster.index]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top_n recommendations
    recommended_products = [products.iloc[i[0]]['name'] for i in sim_scores[1:top_n+1]]
    return recommended_products

# Test the clustered recommendation
clustered_recommendations = get_clustered_recommendations("Organic Cotton T-shirt", cosine_sim, products)
print("Clustered Recommendations for 'Organic Cotton T-shirt':", clustered_recommendations)


Clustered Recommendations for 'Organic Cotton T-shirt': ['Bamboo Cotton T-shirt', 'Recycled Cotton T-shirt', 'Hemp Fabric T-shirt', 'Organic Wool Sweater', 'Organic Cotton Jeans']


# Testing of Clustered Recommendation System

In [12]:
clustered_recommendations = get_clustered_recommendations("Biodegradable Cutlery Set", cosine_sim, products)
print("Clustered Recommendations for 'Biodegradable Cutlery Set':", clustered_recommendations)

Clustered Recommendations for 'Biodegradable Cutlery Set': ['Compostable Dinnerware Set', 'Compostable Travel Mug']


# Hybrid Recommendations

In [13]:
# Example user interaction dataset
interaction_data = pd.DataFrame({
    "user_id": [1, 1, 2, 2, 3],
    "product_id": [1, 2, 3, 1, 4],
    "interaction_type": ["click", "purchase", "click", "add_to_cart", "purchase"]
})

def get_hybrid_recommendations(product_name, cosine_sim, user_id, interaction_data, products, top_n=5):
    # Content-based recommendations
    content_recs = get_recommendations(product_name, cosine_sim, products, top_n=top_n)

    # Collaborative filtering recommendations (user's past interactions)
    user_interactions = interaction_data[interaction_data['user_id'] == user_id]
    cf_recs = user_interactions['product_id'].value_counts().index.tolist()[:top_n]
    cf_recs = products[products['product_id'].isin(cf_recs)]['name'].tolist()

    # Combine and remove duplicates
    combined_recs = list(dict.fromkeys(content_recs + cf_recs))
    return combined_recs[:top_n]

# Test hybrid recommendations
hybrid_recommendations = get_hybrid_recommendations("Organic Cotton T-shirt", cosine_sim, 1, interaction_data, products)
print("Hybrid Recommendations for 'Organic Cotton T-shirt':", hybrid_recommendations)


Hybrid Recommendations for 'Organic Cotton T-shirt': ['Bamboo Cotton T-shirt', 'Recycled Cotton T-shirt', 'Hemp Fabric T-shirt', 'Organic Wool Sweater', 'Organic Cotton Jeans']


# Testing of Hybrid Recommendations System

In [14]:
hybrid_recommendations = get_hybrid_recommendations("Biodegradable Cutlery Set", cosine_sim, 1, interaction_data, products)
print("Hybrid Recommendations for 'Biodegradable Cutlery Set':", hybrid_recommendations)


Hybrid Recommendations for 'Biodegradable Cutlery Set': ['Compostable Dinnerware Set', 'Compostable Travel Mug', 'Recycled Plastic Mug', 'Reclaimed Wood Dining Table', 'Reclaimed Wood Coffee Table']
